In [12]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import label, sum as ndi_sum, center_of_mass
import os
from scipy.ndimage import uniform_filter1d
# --- Main Simulation Function ---
def run_simulation(lam=0.5, zeta=-1.0, R1=10, R2=6, steps= 5000, dt=0.001, save_interval=100):
    np.random.seed(0)

    # Grid setup
    N = 256
    L = 128.0
    dx = L / N
    x = np.arange(N) * dx - L / 2
    y = np.arange(N) * dx - L / 2
    X, Y = np.meshgrid(x, y, indexing="ij")

    kx = np.fft.fftfreq(N, d=dx) * 2 * np.pi
    ky = np.fft.fftfreq(N, d=dx) * 2 * np.pi
    KX, KY = np.meshgrid(kx, ky, indexing="ij")
    K2 = KX**2 + KY**2
    dealias = K2 < (0.67 * K2.max())

    fft = np.fft.fft2
    ifft = lambda f: np.fft.ifft2(f).real

    A, B, K = 1.0, 1.0, 1.0
    sep = 30

    # Initialize two droplets
    phi = -0.9 * np.ones((N, N))
    phi[(X + sep)**2 + Y**2 < R1**2] = 1.0
    phi[(X - sep)**2 + Y**2 < R2**2] = 1.0
    phi += 0.01 * np.random.randn(*phi.shape)
    f_phi = fft(phi)

    radii_history = []

    for step in range(steps + 1):
        phi = np.clip(ifft(f_phi), -2.0, 2.0)

        if not np.isfinite(phi).all():
            print(f"NaN at step {step}")
            break

        if step % save_interval == 0:
            binary = (phi > 0.0).astype(int)
            labeled, num = label(binary)
            areas = ndi_sum(binary, labeled, index=range(1, num + 1))
            COMs = center_of_mass(binary, labeled, index=range(1, num + 1))

            blob_data = [(a, com) for a, com in zip(areas, COMs) if a > 10]
            r_left = r_right = 0.0

            if len(blob_data) >= 1:
                blob_data.sort(key=lambda b: b[1][0])
                if len(blob_data) >= 2:
                    r_left = np.sqrt(blob_data[0][0] / np.pi)
                    r_right = np.sqrt(blob_data[1][0] / np.pi)
                else:
                    x_idx = int(round(blob_data[0][1][0]))
                    x_pos = (x_idx - N // 2) * dx
                    r = np.sqrt(blob_data[0][0] / np.pi)
                    if x_pos < 0:
                        r_left = r
                    else:
                        r_right = r

            radii_history.append((r_left, r_right))
        

        # Compute gradients and update
        dphi_dx = ifft(1j * KX * f_phi)
        dphi_dy = ifft(1j * KY * f_phi)
        grad_phi2 = np.clip(dphi_dx**2 + dphi_dy**2, 0, 100)

        lap_phi = ifft(-K2 * f_phi)
        mu = -A * phi + B * phi**3 - K * lap_phi + lam * grad_phi2
        f_mu = fft(mu)

        Jzeta = 1j * KX * fft(lap_phi * dphi_dx) + 1j * KY * fft(lap_phi * dphi_dy)

        dfdt = -K2 * f_mu - zeta * Jzeta
        dfdt = np.clip(dfdt, -1e4, 1e4)
        dfdt *= dealias
        f_phi += dt * dfdt
        radii_smooth = uniform_filter1d(np.array(radii_history), size=5, axis=0)
    # Analyze growth to classify region
    arr = np.array(radii_history)
    if len(arr) == 0:
        region = "Undetermined"
    else:
        growL = arr[-1, 0] - arr[0, 0]
        growR = arr[-1, 1] - arr[0, 1]
        if growL > 0.2 and growR < -0.2:
            region = "A"  # Forward ripening
        elif growR > 0.2 and growL < -0.2:
            region = "B"  # Reverse ripening
        elif growL > 0.2 and growR > 0.2:
            region = "C"  # Both grow
        else:
            region = "Undetermined"

    print(f"Done: λ={lam}, ζ={zeta} → Region {region}")
    return np.array(radii_history), region


In [ ]:
# Run one simulation
radii, region = run_simulation(
    lam=0.5,
    zeta=-1,
    steps=15000,  # short test
    dt=0.001,    # smaller dt for stability
)


# Plot the result
plt.plot(radii[:, 0], label="Left droplet")
plt.plot(radii[:, 1], label="Right droplet")
plt.xlabel("Time step")
plt.ylabel("Radius")
plt.title(f"λ=0, ζ=0 → Region {region}")
plt.legend()
plt.grid(True)
plt.show()
